<a href="https://colab.research.google.com/github/lvllvl/unet/blob/master/SegmentationModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Overview

## Notes

* imgs/  -- The png image files
* masks/ -- PNG segmentation masks (update these!)
* segs/  -- The outputs in probability from our * * 
internal segnet (unreleased, too big)



###### Categories 
 * 1 - #402020 - road (all parts, anywhere nobody would look at you funny for driving)
 * 2 - #ff0000 - lane markings (don't include non lane markings like turn arrows and crosswalks)
 * 3 - #808060 - undrivable
 * 4 - #00ff66 - movable (vehicles and people/animals)
 * 5 - #cc00ff - my car (and anything inside it, including wires, mounts, etc. No reflections)



## Imports

In [1]:
from pathlib import Path
import pathlib
from PIL import  Image
import cv2
import numpy as np

#### Import U-Net repo

In [2]:
! git clone https://github.com/lvllvl/unet.git

Cloning into 'unet'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 418 (delta 6), reused 0 (delta 0), pack-reused 406
Receiving objects: 100% (418/418), 44.97 MiB | 35.10 MiB/s, done.
Resolving deltas: 100% (92/92), done.


#### import comma10k Dataset 

In [3]:
! git clone https://github.com/commaai/comma10k.git

Cloning into 'comma10k'...
remote: Enumerating objects: 27383, done.
remote: Total 27383 (delta 0), reused 0 (delta 0), pack-reused 27383
Receiving objects: 100% (27383/27383), 3.75 GiB | 42.67 MiB/s, done.
Resolving deltas: 100% (4283/4283), done.
Checking out files: 100% (11203/11203), done.


#### Set up the folders, paths 

In [4]:
path = Path.cwd()

# Comma10k data
comma10kFolder = path/'comma10k'
masksFolder = comma10kFolder/'masks' # Images with the masks ( 5574 images )
imgsFolder = comma10kFolder/'imgs' # Images with no segmentation

# Unet Path 
unetFolder = path/'unet'


# Create directories for Train / validation datasets 
validFiles = comma10kFolder/'valid'
validFiles.mkdir( mode=511, parents= True, exist_ok = False )

trainFiles = comma10kFolder/'train'
trainFiles.mkdir( mode=511, parents=True, exist_ok=False )

# Training folder --> mask folder --> masks.png
# Training folder --> image folder --> image.png
trainMask = trainFiles/'mask'
trainMask.mkdir( mode=511, parents=True, exist_ok = False )

trainImage = trainFiles/'image'
trainImage.mkdir( mode=511, parents=True, exist_ok = False )


# Valid folder --> mask folder --> masks.png
# Valid folder --> image folder --> image.png
validMask = validFiles/'mask'
validMask.mkdir( mode=511, parents=True, exist_ok = False )

validImage = validFiles/'image'
validImage.mkdir( mode=511, parents=True, exist_ok = False )


In [5]:
img = 0
for i in imgsFolder.iterdir():
  name = i.as_uri()[ 7: ]
  print( name )
  img = cv2.imread( name )
  break 
img.shape
# (874, 1164, 3)
# (874, 1164, 3)

/content/comma10k/imgs/0272_55d35794f4955cd1_2018-05-08--18-37-21_13_630.png


(874, 1164, 3)

## Data Preparation

#### Rename files

In [6]:
fileCount = 0
prefix = 'mask_'
suffix = '.png'

for i in masksFolder.iterdir():

  file_path = i.as_uri()[ 7:31 ]
  
  name = i.as_uri()[ 31 : 35 ] 

  newName = file_path + prefix + name + suffix

  i.rename( newName ) # rename the file

  # how to evaluate a model in practice 
  # projects --> github profile, blog, working website 
  # get good at one specific thing --> e.g., computer vision 
  
  fileCount += 1 

  if fileCount % 500 == 0:
    print( 'Still processing...file count = ', fileCount )

Still processing...file count =  500
Still processing...file count =  1000
Still processing...file count =  1500
Still processing...file count =  2000
Still processing...file count =  2500
Still processing...file count =  3000
Still processing...file count =  3500
Still processing...file count =  4000
Still processing...file count =  4500
Still processing...file count =  5000
Still processing...file count =  5500


In [7]:
fileCount = 0
prefix = 'image_'
suffix = '.png'


for i in imgsFolder.iterdir():
  
  file_path = i.as_uri()[ 7:30 ]
  
  name = i.as_uri()[ 30 : 34 ] 

  newName = file_path + prefix + name + suffix

  i.rename( newName ) # rename the file
  fileCount += 1 

  if fileCount % 500 == 0:
    print( 'Still processing...file count = ', fileCount )

Still processing...file count =  500
Still processing...file count =  1000
Still processing...file count =  1500
Still processing...file count =  2000
Still processing...file count =  2500
Still processing...file count =  3000
Still processing...file count =  3500
Still processing...file count =  4000
Still processing...file count =  4500
Still processing...file count =  5000
Still processing...file count =  5500


In [8]:
for i in comma10kFolder.iterdir():
  print( i )

/content/comma10k/scale
/content/comma10k/viewer.py
/content/comma10k/imgs
/content/comma10k/pencil
/content/comma10k/masks
/content/comma10k/.git
/content/comma10k/.gitignore
/content/comma10k/stat.py
/content/comma10k/sample.gif
/content/comma10k/.github
/content/comma10k/README.md
/content/comma10k/sample.jpg
/content/comma10k/valid
/content/comma10k/LICENSE
/content/comma10k/train
/content/comma10k/tools
/content/comma10k/files_trainable
/content/comma10k/requirements.txt


In [9]:
masksFolder.parent

PosixPath('/content/comma10k')

In [10]:
mask = 0 
img = 0 

for i in masksFolder.iterdir():
  mask += 1
for i in imgsFolder.iterdir():
  img += 1

print( mask, img )

5574 5574


#### Split files into train / valid folders

In [11]:
# Masked Files -- Change location 

fileNames = []
count = 0

for i in masksFolder.iterdir():

  fn = i.stem + i.suffix # file name
  # print( 'fn = ', fn ) 

  if count < 4459:  # add to Train folder 
    fileNum = i.stem[ 5: ]
    fileNames.append( fileNum ) # append FileNum to fileNames array 
    
    # Now change the location 
    i.rename( Path( trainMask, fn ))

  else: # add to Valid folder
    i.rename( Path( validFiles, fn ))


  count += 1

print( 'len of fileNames: ', len( fileNames ))
c= 0
for i in trainFiles.iterdir():
  c += 1
print( c )

len of fileNames:  4459
2


In [12]:
# Now move the images from the imgsFolder 
for i in imgsFolder.iterdir():

  fn = i.stem + i.suffix # individual file name 
  
  fileNum = i.stem[ 6: ]

  if fileNum in fileNames: # change location - train folder if fileNum is in array fileNames
    i.rename( Path( trainImage, fn ))

  else: # change location  - valid folder 
    i.rename( Path( validFiles, fn ))

c = 0
for i in trainFiles.iterdir():
  c += 1 
print( c ) 

2


### Data.py

##### Imports, etc

In [13]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

# CommaAI Labels --> 5 classes
road = [ 64,	32,	32 ] 
lane_markings = [ 255, 0, 0 ] 
undriveable = [ 128, 128, 96 ] 
movable = [ 0, 255, 102 ] # vehicles, peoples, animals 
my_vehicle = [ 204,	0, 255 ]

# What does this do?
COLOR_DICT = np.array( [ road, lane_markings, undriveable, movable, my_vehicle ])

##### Function: adjustData

In [14]:
def adjustData( img, mask, flag_multi_class, num_class ):

    if( flag_multi_class ):

        img = img / 255 
        mask = mask[ :,:,:,0 ] if( len( mask.shape ) == 4 ) else mask[ :,:,0 ]
        new_mask = np.zeros( mask.shape + ( num_class, ) )

        for i in range( num_class ):

            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[ mask == i,i ] = 1

        new_mask = np.reshape( new_mask,( new_mask.shape[ 0 ],new_mask.shape[ 1 ]*new_mask.shape[ 2 ],new_mask.shape[ 3 ] )) if flag_multi_class else np.reshape( new_mask,( new_mask.shape[ 0 ] * new_mask.shape[ 1 ],new_mask.shape[ 2 ]))
        mask = new_mask
        
        # TODO: delete this 
        print( 'adjustData: mask.shape --> ' , mask.shape ) 

    elif ( np.max( img ) > 1 ): # if the max value in img is greater than 1

        # make every number img[ i ] 0 <= i <= 1
        img = img / 255
        mask = mask / 255
        mask[ mask > 0.5 ] = 1
        mask[ mask <= 0.5 ] = 0

    return ( img,mask )

##### Function: trainGenerator

In [15]:
def trainGenerator( batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode = "rgb",
                    mask_color_mode = "rgb", image_save_prefix  = "image", mask_save_prefix  = "mask",
                    flag_multi_class = True, num_class = 5, save_to_dir = None, target_size = ( 256,256 ), seed = 1 ):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator( **aug_dict )
    # TODO delete this 
    print( 'image_datagen type : ', type( image_datagen ) )

    mask_datagen = ImageDataGenerator( **aug_dict )
    # TODO delete this 
    print( 'mask_datagen type : ', type( mask_datagen ) )


    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [ image_folder ],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed )
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [ mask_folder ],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed )
    
    train_generator = zip( image_generator, mask_generator )

    for ( img, mask ) in train_generator:

        img, mask = adjustData( img, mask, flag_multi_class, num_class )
        yield ( img, mask )

##### Function: testGenerator 

In [16]:
def testGenerator( test_path, num_image = 30, target_size = ( 256,256 ), flag_multi_class = True, as_gray = False ):
  '''
      num_image -->> ??? What does this do ? total number of images in your test dataset????
      target_size = ( ?, ? ) what should this actually be?
  '''
  for i in range( num_image ): # do the following for all images in test dataset

      img = io.imread( os.path.join( test_path,"%d.png"%i ), as_gray = as_gray )

      img = img / 255

      img = trans.resize( img,target_size )

      img = np.reshape( img, img.shape+(1,) ) if ( not flag_multi_class ) else img

      img = np.reshape( img, (1,) + img.shape )

      yield img

##### Function: geneTrainNpy

In [17]:
def geneTrainNpy( image_path, mask_path, flag_multi_class = True, num_class = 5,
                 image_prefix = "image", mask_prefix = "mask", image_as_gray = False, mask_as_gray = False ):
  '''
  image_path -> path to image
  mask_path -> path to image

  flag_multi_class = T/F --> Are there multiple classes or not?
  num_class = # His is set to 2, because it's black & white 

  image_prefix --> how is filename labeled?
  mask_prefix --> how is filename labeled?

  image_as_gray --> T / F --> is the image gray or not
  mask_as_gray --> T / F --> is MASK gray or not  

  '''
    # Access image path
  image_name_arr = glob.glob( os.path.join( image_path,"%s*.png"%image_prefix ) )

  image_arr = []

  mask_arr = []

  # Loop ... What is index, item? datatype ???
  for index,item in enumerate( image_name_arr ):

      # Open the image, set as grayscale or NOT
      img = io.imread( item, as_gray = image_as_gray )

      # Reshape... only if it's image_as_gray == TRUE 
      img = np.reshape( img,img.shape + (1,) ) if image_as_gray else img

      # open mask image ... wtf does replace do?
      mask = io.imread( item.replace( image_path,mask_path ).replace( image_prefix,mask_prefix ),as_gray = mask_as_gray )

      # Reshape... only if it's mask_as_gray == TRUE 
      mask = np.reshape( mask,mask.shape + (1,) ) if mask_as_gray else mask

      # Call adjustData( ) function 
      img,mask = adjustData( img, mask, flag_multi_class, num_class )

      # append the image, mask to each array         
      image_arr.append( img )
      mask_arr.append( mask )

  image_arr = np.array( image_arr )
  mask_arr = np.array( mask_arr )

  return image_arr, mask_arr

##### Functions: labelVisiualize & saveResult

In [18]:
# What does this do?
def labelVisualize( num_class, color_dict, img ):

    img = img[ :,:,0 ] if len( img.shape ) == 3 else img

    img_out = np.zeros( img.shape + (3,) )

    for i in range( num_class ):
        img_out[ img == i,: ] = color_dict[ i ]

    return img_out / 255




def saveResult( save_path, npyfile, flag_multi_class = False, num_class = 2 ):
  '''
  save_path -->
  npyfile --> 
  flag_multi_class --> bool 
  num_class --> 

  '''

  for i,item in enumerate( npyfile ):

      img = labelVisualize( num_class, COLOR_DICT, item ) if flag_multi_class else item[ :,:,0 ]

      io.imsave( os.path.join( save_path,"%d_predict.png"%i ),img )

## Create the Model

### Model.py

In [19]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

# (874, 1164, 3)
def unet( pretrained_weights = None, input_size = ( 256, 256 ) ):

    inputs = Input( input_size )
    conv1 = Conv2D( 64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal' )( inputs )
    conv1 = Conv2D( 64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal' )( conv1 )
    pool1 = MaxPooling2D( pool_size=( 2, 2 ))( conv1 )
    conv2 = Conv2D( 128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal' )( pool1 )
    conv2 = Conv2D( 128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')( conv2 )
    pool2 = MaxPooling2D( pool_size=( 2, 2 ))( conv2 )
    conv3 = Conv2D( 256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')( pool2 )
    conv3 = Conv2D( 256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')( conv3 )
    pool3 = MaxPooling2D( pool_size=(2, 2))(conv3)
    conv4 = Conv2D( 512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')( pool3 )
    conv4 = Conv2D( 512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')( conv4 )
    drop4 = Dropout( 0.5 )( conv4 )
    pool4 = MaxPooling2D( pool_size=( 2, 2 ))( drop4 )

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    ## upsampling begins here 
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9) # ???? why is (conv9) here?
    

    model = Model(input = inputs, output = conv10)
    # 'categorical_crossentropy'
    # model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

## Execute the model 

### Main.py

In [20]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
trainDir = '/content/comma10k/train'
validDir = '/content/comma10k/valid' 

# Changes to your images 
data_gen_args = dict( rotation_range=0.2,
                      width_shift_range=0.05,
                      height_shift_range=0.05,
                      shear_range=0.05,
                      zoom_range=0.05,
                      horizontal_flip=True,
                      fill_mode='nearest')

# Create a training generator 
myGene = trainGenerator( 100, 
                        trainDir, 
                        'image', 
                        'mask', 
                        data_gen_args, 
                        save_to_dir = None )


# model = unet( pretrained_weights=None, input_size= ( 256, 256, 3 ))
model = unet()


# Save your model at this checkpoint 
# MCP( filepath, 
#      monitor = the value being monitored, e.g., 'loss' 
#      verbose = 0: shows you nothing, 1: shows animated progress bar, 2: shows # of epochs 1/10  
#      save_best_only= If true, then model weights are saved)
model_checkpoint = ModelCheckpoint('content/comma10k/unet_CommaModel.hdf5', 
                                   monitor = 'loss',
                                   verbose = 2, 
                                   save_best_only = True)


model.fit_generator( myGene,
                    steps_per_epoch = 300,
                    epochs=1,
                    callbacks=[ model_checkpoint ])

testGene = testGenerator( validDir )

results = model.predict_generator( testGene,
                                   30,
                                   verbose=1 )

# Save your results HERE
saveResult( validDir,
            results )

ValueError: ignored

## Save Model, Save Predictor

In [ ]:
# Save model, save as a predictor

In [ ]:
for i in comma10kFolder.iterdir():
  print( i )